# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [15]:
# Your code here
import pandas as pd
import sqlite3

con = sqlite3.connect('data.sqlite')


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [16]:
# Your code here
pd.read_sql('''
SELECT employees.firstName, employees.lastName, offices.city
FROM employees
    JOIN offices
    ON employees.officeCode = offices.officeCode
WHERE offices.city = 'Boston'
''', con)

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [18]:
# Your code here
pd.read_sql('''
SELECT off.officeCode, off.city, COUNT(em.employeeNumber) AS number_of_employees
FROM offices AS off
    LEFT JOIN employees AS em
    USING(officeCode)
GROUP BY officeCode
HAVING number_of_employees = 0
''', con)

,officeCode,city,number_of_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [40]:
# Answers will vary

# Example question: How many customers are there per office?
pd.read_sql('''
SELECT off.officeCode, off.city, COUNT(cus.customerNumber) AS number_of_customers
FROM offices AS off
    JOIN employees AS em
    USING(officeCode)
        JOIN customers AS cus
        ON em.employeeNumber = cus.salesRepEmployeeNumber
GROUP BY off.officeCode
''', con)

,officeCode,city,number_of_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [48]:
# How much did each employee sold? (maybe the top 10)

pd.read_sql('''
SELECT em.employeeNumber, em.firstName, em.lastName, CAST(SUM(pay.amount) AS int) as Amount_sold
FROM employees AS em
    JOIN customers as cus
    ON em.employeeNumber = salesRepEmployeeNumber
        JOIN payments as pay
        USING(customerNumber)
GROUP BY em.employeeNumber
ORDER BY Amount_sold DESC
LIMIT 10
''',con)

,employeeNumber,firstName,lastName,Amount_sold
0,1370,Gerard,Hernandez,1112003
1,1165,Leslie,Jennings,989906
2,1401,Pamela,Castillo,750201
3,1501,Larry,Bott,686653
4,1504,Barry,Jones,637672
5,1323,George,Vanauf,584406
6,1337,Loui,Bondur,569485
7,1611,Andy,Fixter,509385
8,1612,Peter,Marsh,497907
9,1286,Foon Yue,Tseng,488212


In [69]:
# Top product sold in a single city

pd.read_sql('''
SELECT cus.city, pr.productCode, pr.productName, COUNT(pr.productCode) AS Amount_sold
FROM products AS pr
    JOIN orderdetails AS od
    USING (productCode)
        JOIN orders AS o
        USING(orderNumber)
            JOIN customers AS cus
            USING (customerNumber)           
GROUP BY cus.city
ORDER BY Amount_sold DESC
LIMIT 1
''', con)

# Your code here

,city,productCode,productName,Amount_sold
0,Madrid,S10_1678,1969 Harley Davidson Ultimate Chopper,304


In [ ]:
"""
Question 3
"""

# Your code here

## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [72]:
# Your code here

pd.read_sql('''
SELECT pr.productCode, pr.productName, em.firstName AS employee_name, em.lastName AS employee_lastName
FROM products AS pr
    JOIN orderdetails AS od
    USING (productCode)
        JOIN orders AS o
        USING(orderNumber)
            JOIN customers AS cus
            USING (customerNumber)  
                JOIN employees as em
                ON cus.salesRepEmployeeNumber = em.employeeNumber
''', con)


,productCode,productName,employee_name,employee_lastName
0,S10_1678,1969 Harley Davidson Ultimate Chopper,George,Vanauf
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Loui,Bondur
2,S10_1678,1969 Harley Davidson Ultimate Chopper,Loui,Bondur
3,S10_1678,1969 Harley Davidson Ultimate Chopper,Leslie,Thompson
4,S10_1678,1969 Harley Davidson Ultimate Chopper,Leslie,Jennings
...,...,...,...,...
2991,S72_3212,Pont Yacht,Peter,Marsh
2992,S72_3212,Pont Yacht,Larry,Bott
2993,S72_3212,Pont Yacht,Gerard,Hernandez
2994,S72_3212,Pont Yacht,Gerard,Hernandez


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [73]:
# Your code here
pd.read_sql('''
SELECT firstName, lastName, SUM(quantityOrdered) as total_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
ORDER BY lastName
''', con)

,firstName,lastName,total_products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [74]:
# Your code here
pd.read_sql('''
SELECT firstName, lastName, COUNT(productCode) as different_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
HAVING different_products_sold > 200
ORDER BY lastName

''', con)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!